In [1]:
import random
from pathlib import Path
from openai import OpenAI
from config import api_key
from utils import load_dataset, create_dataset
from pprint import pprint

In [2]:
client = OpenAI(api_key=api_key)

results_path = Path.cwd() / "results"
if not results_path.exists():
    results_path.mkdir()

In [3]:
dataset, tag_counts = load_dataset("capital.json")
print(len(dataset))
pprint(tag_counts)

200
{'all_lower': 67,
 'all_upper': 59,
 'city': 25,
 'even_words': 97,
 'fragment_subject': 20,
 'fragment_verb': 20,
 'gpt4': 200,
 'nature': 62,
 'odd_words': 103,
 'proper_noun': 32,
 'sentence': 160,
 'shared_sample': 42,
 'start_cap': 133,
 'start_lower': 67,
 'start_the': 57}


In [4]:
# datasets: all upper vs lower; starts with a capital vs not; has a capital vs not
# even vs odd counts; even vs odd length w/ all upper vs lower-->even mixed, odd mixed
# fragments vs sentence; just subject/verb fragments vs all
# nature vs rest
# contains a proper noun
tags_true = ["all_upper"]
tags_false = ["all_lower"]
tags_common = ["sentence"]
positives, negatives = create_dataset(dataset, tags_true, tags_false, tags_common)

In [5]:
pprint(positives)

[{'input': 'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'nature',
           'odd_words',
           'start_the',
           'gpt4']},
 {'input': 'TODAY IS A BEAUTIFUL DAY.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'odd_words',
           'gpt4']},
 {'input': 'I LOVE TO TRAVEL AND EXPLORE NEW PLACES.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'even_words',
           'gpt4']},
 {'input': 'PYTHON IS A POPULAR PROGRAMMING LANGUAGE.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
           'even_words',
           'gpt4',
           'proper_noun']},
 {'input': 'THE SUN SETS IN THE WEST.',
  'tags': ['start_cap',
           'all_upper',
           'sentence',
           'shared_sample',
  

In [6]:
pprint(negatives)

[{'input': 'the quick brown fox jumps over the lazy dog.',
  'tags': ['shared_sample',
           'sentence',
           'nature',
           'odd_words',
           'start_the',
           'all_lower',
           'start_lower',
           'gpt4']},
 {'input': 'today is a beautiful day.',
  'tags': ['all_lower',
           'shared_sample',
           'sentence',
           'nature',
           'odd_words',
           'start_lower',
           'gpt4']},
 {'input': 'i love to travel and explore new places.',
  'tags': ['all_lower',
           'shared_sample',
           'sentence',
           'even_words',
           'start_lower',
           'gpt4']},
 {'input': 'python is a popular programming language.',
  'tags': ['all_lower',
           'shared_sample',
           'sentence',
           'even_words',
           'start_lower',
           'gpt4',
           'proper_noun']},
 {'input': 'the sun sets in the west.',
  'tags': ['shared_sample',
           'sentence',
           'nature',


In [7]:
length_range = list(range(1, 21))

def generate_sample(f_condition):
    string_len = random.choice(length_range)
    output_string = ""
    while len(output_string.split()) < string_len:
        sample = random.choice(words)
        if f_condition(sample):
            output_string += " " + sample
        output_string = output_string.strip()
    return output_string

def append_example(prompt, f_cond_true, n=1):
    f_cond_false = lambda x: not f_cond_true(x)
    for _ in range(n):
        sample_true = generate_sample(f_cond_true)
        sample_false = generate_sample(f_cond_false)
        prompt += f"\nInput: \"{sample_true}\"\nLabel: True\n"
        prompt += f"\nInput: \"{sample_false}\"\nLabel: False\n"
    return prompt

def gen_user_query(f_cond_true, n=1):
    f_cond_false = lambda x: not f_cond_true(x)
    labels = []
    output_string = ""
    for _ in range(n):
        labels.append(random.choice([True, False]))
        sample = generate_sample(f_cond_true if labels[-1] else f_cond_false)
        output_string += f"\nInput: \"{sample}\"\nLabel:\n"
    return output_string, labels

In [8]:
file_name = "random_even_letter.txt"
rule_name = "Random String: Even Number of Letters"

def write_header(x):
    line_delim = ["="] * 79
    line_delim = "".join(line_delim)
    return f"{line_delim}\n{x}\n{line_delim}\n"

rule = "The input string has word of even length."
test_true = lambda x: len(x) % 2 == 0

system_prompt = """
Below are examples from a string classification task.
You will be asked by the user to classify new input strings along with other queries pertaining to the classification task.
Each input is classified as either True or False according to a rule that can be stated simply in natural language.

"""

system_prompt = append_example(system_prompt, test_true, n=10)

system_prompt += """
When responding to user queries, you should respond in the format above, with the input string in quotes and the label as either True or False.
"""

print(system_prompt)

with open(results_path / file_name, "w") as f:
    f.write(write_header(rule_name))
    f.write(f"\nRULE: {rule}\n\n")

with open(results_path / file_name, "a") as f:
    f.write(write_header("SYSTEM PROMPT"))
    f.write(system_prompt)
    f.write("\n")
    f.write(write_header("RESULTS"))

NameError: name 'words' is not defined

In [ ]:
def gpt_prediction(system_prompt, user_query, model="gpt-4"):
  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": system_prompt
      },
      {
        "role": "user",
        "content": user_query
      }
    ],
    temperature=1,
    max_tokens=len(system_prompt),
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [ ]:
output_string, labels = gen_user_query(test_true, n=10)
user_query = output_string
print(gpt_prediction(system_prompt, user_query))

Input: "splenunculus aistopod preday compulsative"
Label: True

Input: "nephrotoxicity remantle yields dhai semiaxis colourableness wooliest sororize amylogenic examiner gour disparages compenetrate shattering"
Label: False

Input: "bachelorette embace wordbook hollowed wittolly endosiphonal oversolidified hydrothermal degenerative"
Label: True

Input: "unsuperfluousness monsoon chrysanisic sheerly bibbled byblidaceae bimasty curiosity avenses"
Label: False

Input: "unconvertible successlessly pulchritude sulphurated ingates unhorny dynodes polyaffectioned immortalising lemogra veering recharged arousal topsy preadvise myoinositol phymatoid tepache"
Label: False

Input: "aguacateca trophodisc sociocentric hypothecater unappreciatively lamnidae negligibleness misleading endaze inscript expediteness misenite fluorotype merrytrotter legend camerine despised samucu stainabilities lurching"
Label: False

Input: "chesses unslacked sab cooja soaringly pococurantism photochronography impoveris

In [ ]:
print(labels)

[True, True, True, False, False, True, False, True, True, True]


In [ ]:
split_query = user_query.split("\n")
queries = [split_query[i] for i in range(len(split_query)) if i % 3 == 1]
print(len(queries))
for label, query in zip(labels, queries):
    print(f"{query}\nLabel: {label}")
    query = f"{query}\nLabel:"
    print(f"MODEL: {gpt_prediction(system_prompt, query)}")
    print()

10
Input: "oogamies fluorboric zees lemography waggoned meow slaver sandmite stroky expression unrummaged episodes archaeologic cheeringly unsturdily victrola tyrannidae discophore"
Label: True
MODEL: False

Input: "everlasting protuberate"
Label: False
MODEL: True

Input: "garrulinae anogenital"
Label: True
MODEL: True

Input: "predissolve phosphomolybdic crotons woodchats"
Label: False


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-AyNKjYMIA668Sc8nezuCmdTx on tokens per min (TPM): Limit 40000, Used 37277, Requested 3761. Please try again in 1.557s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}